# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
# city_data_df.count()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,19.80,84,0,2.57,US,1696294341
1,1,san patricio,28.0170,-97.5169,29.35,81,0,6.08,US,1696294403
2,2,new norfolk,-42.7826,147.0587,20.72,81,76,4.85,AU,1696294404
3,3,waitangi,-43.9535,-176.5597,8.79,93,100,6.26,NZ,1696294404
4,4,olonkinbyen,70.9221,-8.7187,2.97,84,79,7.29,SJ,1696294404


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_cities = city_data_df.hvplot.points("Lat", 
                                        "Lng", 
                                        geo = True,
                                        color = "b",
                                        alpha=0.25, 
                                        xlim=(-200, 200), 
                                        ylim=(-100, 100),
                                        frame_height = 450,
                                        tiles = "OSM")

# Display the map
map_cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
    #My ideal weather condition is 
    #   -temperatures between 20 to 25 fahrenheit.
    #   -temperatures cloudiness up to 2.
    #   -temperatures wind speed up to 5.
    #   -temperatures humidity up to 70.

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27) 
                                    & (city_data_df["Max Temp"] >= 20)
                                    & (city_data_df["Cloudiness"] <= 2) 
                                    & (city_data_df["Wind Speed"] <= 5) 
                                    & (city_data_df["Humidity"] <= 80)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
# print(city_data_df.count())
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,ha'il,27.5219,41.6907,25.01,33,0,2.06,SA,1696294409
33,33,hamilton,39.1834,-84.5333,23.88,71,0,0.45,US,1696294008
36,36,fort bragg,35.1390,-79.0060,22.75,76,0,2.06,US,1696294282
38,38,saint-pierre,-21.3393,55.4781,21.82,64,0,3.60,RE,1696294416
97,97,dadhar,29.4753,67.6514,26.73,10,0,2.80,PK,1696294438
145,145,tumayr,25.7039,45.8680,26.95,18,0,3.67,SA,1696294456
215,215,port alfred,-33.5906,26.8910,20.23,54,0,2.78,ZA,1696294483
250,250,stanthorpe,-28.6667,151.9500,24.88,22,0,3.95,AU,1696294494
258,258,afif,23.9065,42.9172,24.54,20,0,4.05,SA,1696294498
262,262,graaff-reinet,-32.2522,24.5308,22.15,29,0,3.62,ZA,1696294499


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.count()
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,ha'il,SA,27.5219,41.6907,33,
33,hamilton,US,39.1834,-84.5333,71,
36,fort bragg,US,35.1390,-79.0060,76,
38,saint-pierre,RE,-21.3393,55.4781,64,
97,dadhar,PK,29.4753,67.6514,10,
145,tumayr,SA,25.7039,45.8680,18,
215,port alfred,ZA,-33.5906,26.8910,54,
250,stanthorpe,AU,-28.6667,151.9500,22,
258,afif,SA,23.9065,42.9172,20,
262,graaff-reinet,ZA,-32.2522,24.5308,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel","apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.count()
hotel_df

Starting hotel search
ha'il - nearest hotel: TE ORA Bag Evi Butik Hotel
hamilton - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
dadhar - nearest hotel: No hotel found
tumayr - nearest hotel: Phoenix panzió
port alfred - nearest hotel: No hotel found
stanthorpe - nearest hotel: No hotel found
afif - nearest hotel: Spark hotel
graaff-reinet - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
pyrgos - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found
dalianwan - nearest hotel: No hotel found
mardan - nearest hotel: No hotel found
nola - nearest hotel: No hotel found
tochio-honcho - nearest hotel: No hotel found
santanyi - nearest hotel: No hotel found
igoumenitsa - nearest hotel: No hotel found
kotkapura - nearest hotel: No hotel found
fes al bali - nearest hotel: No hotel found
iwaki - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
19,ha'il,SA,27.5219,41.6907,33,TE ORA Bag Evi Butik Hotel
33,hamilton,US,39.1834,-84.5333,71,No hotel found
36,fort bragg,US,35.1390,-79.0060,76,No hotel found
38,saint-pierre,RE,-21.3393,55.4781,64,No hotel found
97,dadhar,PK,29.4753,67.6514,10,No hotel found
145,tumayr,SA,25.7039,45.8680,18,Phoenix panzió
215,port alfred,ZA,-33.5906,26.8910,54,No hotel found
250,stanthorpe,AU,-28.6667,151.9500,22,No hotel found
258,afif,SA,23.9065,42.9172,20,Spark hotel
262,graaff-reinet,ZA,-32.2522,24.5308,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points("Lat", 
                                    "Lng", 
                                    geo = True,
                                    color = "b",
                                    alpha=0.25, 
                                    xlim=(-200, 200), 
                                    ylim=(-100, 100),
                                    frame_height = 450,
                                    tiles = "OSM",
                                    hover_cols=["Hotel Name", "Country"])

# Display the map
map_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (Hotel Name,Country)